In [ ]:
# Install libraries if not already installed
!pip install -q earthengine-api geemap

import ee
import geemap
import pandas as pd
import numpy as np

# Authenticate and Initialize
try:
    ee.Initialize(project='dojo-485716')
except:
    ee.Authenticate()
    ee.Initialize(project='dojo-485716')


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 10.1 MB/s eta 0:00:00


In [ ]:
farmer_portfolio = {
    "George Mukama": [
        {
            "crop": "Maize", "size": 10,
            "coords": [[1.11739,33.90203], [1.11665,33.90119], [1.11615,33.90157], [1.11662,33.90255]]
        },
        {
            "crop": "Avocado", "size": 5,
            "coords": [[1.11726,33.9014], [1.11713,33.90152], [1.11688,33.90118], [1.11699,33.90109]]
        },
        {
            "crop": "Eucalyptus", "size": 15,
            "coords": [[1.117927, 33.901234], [1.117563, 33.901470], [1.117906, 33.900633], [1.117906, 33.901244]]
        }
    ],
    "Tukule Samuel": [
        {
            "crop": "Coffee", "size": 4,
            "coords": [[1.078478, 33.884497], [1.079439, 33.884931], [1.080126, 33.884920], [1.079413, 33.885747]]
        },
        {
            "crop": "Cassava", "size": 1,
            "coords": [[1.08106,33.88796], [1.08084,33.88807], [1.08071,33.8878], [1.08095,33.88773]]
        },
        {
            "crop": "Maize", "size": 0.5,
            "coords": [[1.079965, 33.886170], [1.08003, 33.886905], [1.079590, 33.886916], [1.079568, 33.88615]]
        }
    ],
    "Emmanuel Gonahasa": [
        {
            "crop": "Avocado", "size": 5,
            "coords": [[1.05659,33.87743], [1.05638,33.87717], [1.05607,33.87747], [1.05619,33.8776]]
        },
        {
            "crop": "Eucalyptus", "size": 12,
            "coords": [[1.057428, 33.879250], [1.057396, 33.880495], [1.056655, 33.880527], [1.056022, 33.879218]]
        },
        {
            "crop": "Maize", "size": 5,
            "coords": [[1.054167, 33.880752], [1.052815, 33.880817], [1.052601, 33.882553], [1.053995, 33.882769]]
        }
    ],
    "Noah Natude": [
        {
            "crop": "Tomatoes", "size": 8,
            "coords": [
                [1.070147, 33.885127], [1.069056, 33.885576], [1.068861, 33.886234],
                [1.069517, 33.886981], [1.070826, 33.886032]
            ]
        },
        {
            "crop": "Rice", "size": 30,
            "coords": [
                [1.066624, 33.886600], [1.066857, 33.885685], [1.067888, 33.885577],
                [1.067928, 33.885910], [1.067411, 33.886023], [1.067481, 33.886364],
                [1.068000, 33.886237], [1.068149, 33.886648], [1.067708, 33.886846]
            ]
        },
        {
            "crop": "Groundnuts", "size": 8,
            "coords": [
                [1.070147, 33.885127], [1.069056, 33.885576], [1.068861, 33.886234],
                [1.069517, 33.886981], [1.070826, 33.886032]
            ]
        }
    ],
    "James Balya": [
        {
            "crop": "Oranges", "size": 2.5,
            "coords": [
                [0.982982, 33.857992], [0.982250, 33.858040],[0.982145, 33.858709], [0.983219, 33.858857], [0.983246, 33.858403], [0.982827, 33.858311]

            ]
        }
    ],
    "Cornelius Kaberwa": [
        {
            "crop": "Rice", "size": 3,
            "coords": [[1.117927, 33.901234], [1.117563, 33.901470], [1.117638, 33.900633], [1.117906, 33.901244]]
        }
    ]
}

In [ ]:
def analyze_complete_farmer_holdings(farmer_name, start_date, end_date, grid_size=20):
    if farmer_name not in farmer_portfolio:
        return "Farmer not found."

    print(f"📊 Starting Full Analysis for {farmer_name}...")
    Map = geemap.Map()

    # Storage for dataframe
    all_records = []

    # 1. Identify Unique Parcels
    # (We group by coordinates to avoid double-counting area for seasonal crops)
    unique_parcels = {}
    for farm in farmer_portfolio[farmer_name]:
        coord_tuple = tuple(tuple(c) for c in farm['coords']) # Make coordinates hashable
        if coord_tuple not in unique_parcels:
            unique_parcels[coord_tuple] = {'crops': [farm['crop']], 'size_reported': farm['size']}
        else:
            if farm['crop'] not in unique_parcels[coord_tuple]:
                unique_parcels[coord_tuple]['crops'].append(farm['crop'])

    # Prepare for Map Zoom
    all_geoms = []

    # 2. Process Each Unique Parcel
    parcel_idx = 1
    for coords, info in unique_parcels.items():
        # Setup Geometry
        poly = get_farm_geometry(list(coords))
        all_geoms.append(ee.Feature(poly))

        # Calculate Exact Satellite-Derived Acreage
        area_acres = poly.area().divide(4046.86).getInfo()
        crops_str = " & ".join(info['crops'])
        print(f"📍 Parcel {parcel_idx}: {crops_str} | Calculated Area: {area_acres:.2f} Acres")

        # 3. Create Sub-Plot Grid for this Parcel
        grid = poly.coveringGrid(poly.projection(), grid_size).filterBounds(poly)

        # 4. Get Satellite Imagery & Indices (Median for the requested period)
        s2 = (ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')
              .filterBounds(poly).filterDate(start_date, end_date)
              .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 40))
              .median())

        if s2.bandNames().size().getInfo() > 0:
            ndvi = s2.normalizedDifference(['B8', 'B4']).rename('NDVI')
            ndwi = s2.normalizedDifference(['B8', 'B11']).rename('NDWI')
            bsi = s2.expression('((B11 + B4) - (B8 + B2)) / ((B11 + B4) + (B8 + B2))',
                               {'B11': s2.select('B11'), 'B8': s2.select('B8'),
                                'B4': s2.select('B4'), 'B2': s2.select('B2')}).rename('BSI')

            combined = ee.Image.cat([ndvi, ndwi, bsi])

            # 5. Climate Data (Rainfall)
            rain = (ee.ImageCollection('UCSB-CHG/CHIRPS/DAILY')
                   .filterBounds(poly).filterDate(start_date, end_date)
                   .sum().reduceRegion(ee.Reducer.mean(), poly, 5000).getInfo().get('precipitation'))

            # 6. Extract Data for DataFrame
            stats = combined.reduceRegions(collection=grid, reducer=ee.Reducer.mean(), scale=10).getInfo()
            for feature in stats['features']:
                results = feature['properties']
                all_records.append({
                    'Farmer': farmer_name,
                    'Parcel_ID': f"Farm_{parcel_idx}",
                    'Crops': crops_str,
                    'Acreage': round(area_acres, 2),
                    'NDVI_Health': results.get('NDVI'),
                    'NDWI_Water': results.get('NDWI'),
                    'BSI_Soil': results.get('BSI'),
                    'Period_Rain_mm': rain
                })

            # 7. Add to Map
            # NDVI Visual Layer
            ndvi_vis = {'min': 0, 'max': 0.8, 'palette': ['#d73027', '#f46d43', '#fee08b', '#d9ef8b', '#1a9850']}
            Map.addLayer(ndvi.clip(poly), ndvi_vis, f"Health: {crops_str}")

            # Boundary Layer with Label
            Map.addLayer(ee.Image().paint(poly, 0, 2), {'palette': 'white'}, f"Boundary: Farm {parcel_idx}")

        parcel_idx += 1

    # 8. Create the Legend (Key)
    legend_dict = {
        'Excellent Health (High NDVI)': '#1a9850',
        'Good Health': '#a6d96a',
        'Moderate/Stressed': '#fee08b',
        'Poor/Bare Soil': '#d73027',
        'Farm Boundary': '#ffffff'
    }
    Map.add_legend(title=f"Legend: {farmer_name}'s Farms", legend_dict=legend_dict)

    # Final Map Setup
    Map.centerObject(ee.FeatureCollection(all_geoms), 15)

    return pd.DataFrame(all_records), Map



In [ ]:
def get_farm_geometry(coords):
    """Helper to convert [Lat, Lon] list to EE Polygon [Lon, Lat]"""
    return ee.Geometry.Polygon([[p[1], p[0]] for p in coords])

def extract_high_res_historical(farmer_name, step_days=10):
    if farmer_name not in farmer_portfolio: return None

    print(f"⌛ Processing Calendar-Correct Timeline for {farmer_name}...")
    results = []

    # Generate a list of valid calendar dates using Pandas
    # This automatically handles Leap Years and different month lengths
    date_series = pd.date_range(start='2020-01-01', end='2025-01-01', freq=f'{step_days}D')

    for farm in farmer_portfolio[farmer_name]:
        crop = farm['crop']
        poly_coords = [[p[1], p[0]] for p in farm['coords']]
        geom = ee.Geometry.Polygon(poly_coords)

        for i in range(len(date_series) - 1):
            # Get the start and end of the window from our valid calendar list
            p_start_py = date_series[i]
            p_end_py = date_series[i+1]

            # Format for the CSV
            date_label = p_start_py.strftime('%Y-%m-%d')

            # Convert to Earth Engine Dates for the filters
            p_start_ee = ee.Date(date_label)
            p_end_ee = ee.Date(p_end_py.strftime('%Y-%m-%d'))

            # --- 1. NDVI (Sentinel-2) ---
            ndvi_coll = (ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')
                      .filterBounds(geom)
                      .filterDate(p_start_ee, p_end_ee)
                      .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 50)))

            ndvi_val = None
            if ndvi_coll.size().getInfo() > 0:
                ndvi_img = ndvi_coll.median().normalizedDifference(['B8', 'B4'])
                stats = ndvi_img.reduceRegion(ee.Reducer.mean(), geom, 10).getInfo()
                ndvi_val = stats.get('nd')

            # --- 2. Rainfall (CHIRPS) ---
            rain_img = (ee.ImageCollection('UCSB-CHG/CHIRPS/DAILY')
                       .filterBounds(geom)
                       .filterDate(p_start_ee, p_end_ee)
                       .sum())

            rain_stats = rain_img.reduceRegion(ee.Reducer.mean(), geom, 5000).getInfo()
            rain_val = rain_stats.get('precipitation')

            results.append({
                'Farmer': farmer_name,
                'Crop': crop,
                'Start_Date': date_label,
                'NDVI_Health': ndvi_val,
                'Rainfall_mm': rain_val
            })

        print(f"  ✅ Completed {crop}")

    return pd.DataFrame(results)

def get_rainfall_analysis(farmer_name):
    """Provides a current rainfall risk summary"""
    print(f"🌦️ Analyzing Current Rainfall: {farmer_name}")
    geom = get_farm_geometry(farmer_portfolio[farmer_name][0]['coords'])
    rain_sum = ee.ImageCollection('UCSB-CHG/CHIRPS/DAILY').filterDate('2024-12-01', '2025-01-30').sum().reduceRegion(ee.Reducer.mean(), geom.centroid(), 5000).getInfo().get('precipitation')

    if rain_sum is None: return "Data unavailable"

    status = "✅ OPTIMAL"
    if rain_sum < 60: status = "🚨 SEVERE DROUGHT - Action Required"
    elif rain_sum < 100: status = "⚠️ MILD STRESS - Monitor"
    elif rain_sum > 250: status = "🌊 FLOOD RISK"

    return f"Total Rain (Dec-Jan): {rain_sum:.1f}mm | Status: {status}"

In [ ]:
# This will show Noah's Rice (30 acres) and his combined Tomato/Groundnut plot (8 acres)
tukule_results_df, tukule_map = analyze_complete_farmer_holdings("Tukule Samuel", "2024-12-01", "2025-01-30")

# Show the interactive map
tukule_map

📊 Starting Full Analysis for Tukule Samuel...
📍 Parcel 1: Coffee | Calculated Area: 2.07 Acres
📍 Parcel 2: Cassava | Calculated Area: 0.21 Acres
📍 Parcel 3: Maize | Calculated Area: 0.96 Acres


Map(center=[1.0795701722859952, 33.88571740908405], controls=(WidgetControl(options=['position', 'transparent_…

In [ ]:

george_results_df, george_map = analyze_complete_farmer_holdings("George Mukama", "2024-12-01", "2025-01-30")


george_map

📊 Starting Full Analysis for George Mukama...
📍 Parcel 1: Maize | Calculated Area: 2.60 Acres
📍 Parcel 2: Avocado | Calculated Area: 0.20 Acres
📍 Parcel 3: Eucalyptus | Calculated Area: 0.32 Acres


Map(center=[1.1168566942264835, 33.901749742041254], controls=(WidgetControl(options=['position', 'transparent…

In [ ]:
def generate_daily_scouting_snapshot(farmer_name, target_date, grid_size=30):
    if farmer_name not in farmer_portfolio:
        return "Farmer not found."

    # Convert string date to EE date
    date_obj = ee.Date(target_date)

    # Define a search window (Target Date minus 5 days)
    # This ensures we catch the most recent satellite pass
    start_search = date_obj.advance(-5, 'day')
    end_search = date_obj.advance(1, 'day')

    # Identify Unique Land Parcels
    seen_coords = set()
    parcels = [f for f in farmer_portfolio[farmer_name] if not (tuple(tuple(c) for c in f['coords']) in seen_coords or seen_coords.add(tuple(tuple(c) for c in f['coords'])))]

    for idx, parcel in enumerate(parcels):
        poly = get_farm_geometry(parcel['coords'])

        # 1. FIND THE SPECIFIC IMAGE
        # We sort by 'CLOUD_COVER' so if there are two passes, we get the clearest one
        s2_collection = (ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')
                        .filterBounds(poly)
                        .filterDate(start_search, end_search)
                        .sort('CLOUDY_PIXEL_PERCENTAGE'))

        image_count = s2_collection.size().getInfo()

        if image_count == 0:
            print(f"❌ No satellite data found for {farmer_name} near {target_date}. Try a different date.")
            continue

        # Pick the best single image from that window
        best_image = ee.Image(s2_collection.first())
        actual_date = ee.Date(best_image.get('system:time_start')).format('YYYY-MM-DD').getInfo()
        cloud_pct = best_image.get('CLOUDY_PIXEL_PERCENTAGE').getInfo()

        print(f"\n{'='*60}")
        print(f"📸 SNAPSHOT REPORT: {farmer_name} | FARM {idx+1}")
        print(f"📅 Requested: {target_date} | 🛰️ Found Image: {actual_date}")
        print(f"☁️ Cloud Cover: {cloud_pct:.1f}%")
        print(f"{'='*60}")

        # Calculate Indices
        ndvi = best_image.normalizedDifference(['B8', 'B4']).rename('NDVI')
        ndwi = best_image.normalizedDifference(['B8', 'B11']).rename('NDWI')
        bsi = best_image.expression('((B11 + B4) - (B8 + B2)) / ((B11 + B4) + (B8 + B2))',
                                   {'B11': best_image.select('B11'), 'B8': best_image.select('B8'),
                                    'B4': best_image.select('B4'), 'B2': best_image.select('B2')}).rename('BSI')

        # Create Numbered Grid
        grid = poly.coveringGrid(poly.projection(), grid_size).filterBounds(poly)

        # Generate Maps (Showing NDVI as example - you can repeat for NDWI/BSI)
        m = geemap.Map()
        m.centerObject(poly, 18)

        # Add the Indices
        m.addLayer(ndvi.clip(poly), {'min': 0.1, 'max': 0.8, 'palette': ['red', 'yellow', 'green']}, 'Health (NDVI)')
        m.addLayer(ndwi.clip(poly), {'min': -0.1, 'max': 0.5, 'palette': ['brown', 'yellow', 'blue']}, 'Water (NDWI)')

        # Add the Scouting Grid
        m.addLayer(grid, {'color': 'white'}, 'Scouting Grid')

        # Add Labels to identify the date clearly on the map
        m.add_text(f"Snapshot: {actual_date}", position='topright')

        display(m)

# --- EXECUTION ---
# Example: Look for an image on Christmas Day 2024
generate_daily_scouting_snapshot("Noah Natude", "2024-12-25")


📸 SNAPSHOT REPORT: Noah Natude | FARM 1
📅 Requested: 2024-12-25 | 🛰️ Found Image: 2024-12-359
☁️ Cloud Cover: 66.8%


Map(center=[1.0697739145988707, 33.88601309851582], controls=(WidgetControl(options=['position', 'transparent_…


📸 SNAPSHOT REPORT: Noah Natude | FARM 2
📅 Requested: 2024-12-25 | 🛰️ Found Image: 2024-12-359
☁️ Cloud Cover: 66.8%


Map(center=[1.067349474706878, 33.88623534356942], controls=(WidgetControl(options=['position', 'transparent_b…

In [ ]:
# 1. Run Map for a specific farmer
analyze_farmer_portfolio("Emmanuel Gonahasa")

📊 Mapping Distinct Plots for: Emmanuel Gonahasa
📍 Mapped separate Avocado plot (5 acres)
📍 Mapped separate Eucalyptus plot (12 acres)
📍 Mapped separate Maize plot (5 acres)


Map(center=[1.0546483177376071, 33.8809689443822], controls=(WidgetControl(options=['position', 'transparent_b…

In [ ]:


# 2. Generate and Download 10-Day Report
noah_report = extract_high_res_historical("Noah Natude")
noah_report.to_csv("Noah_10Day_Analysis.csv", index=False)
# files.download("Noah_10Day_Analysis.csv") # Uncomment to auto-download

# 3. Quick Weather Check
print(get_rainfall_analysis("Noah Natude"))

⌛ Processing Calendar-Correct Timeline for Noah Natude...
  ✅ Completed Tomatoes
  ✅ Completed Rice
  ✅ Completed Groundnuts
🌦️ Analyzing Current Rainfall: Noah Natude
Total Rain (Dec-Jan): 69.7mm | Status: ⚠️ MILD STRESS - Monitor


In [ ]:
noah_data = extract_high_res_historical("Noah Natude")
df_noah = pd.DataFrame(noah_data)
# df_noah.to_csv("Noah_10Day_Analysis.csv")

⌛ Processing Calendar-Correct Timeline for Noah Natude...
  ✅ Completed Tomatoes
  ✅ Completed Rice
  ✅ Completed Groundnuts


In [ ]:
print(df_noah.head())

        Farmer      Crop  Start_Date  NDVI_Health  Rainfall_mm
0  Noah Natude  Tomatoes  2020-01-01     0.374877     7.329374
1  Noah Natude  Tomatoes  2020-01-11     0.410368     0.000000
2  Noah Natude  Tomatoes  2020-01-21     0.177183    65.002005
3  Noah Natude  Tomatoes  2020-01-31     0.530142    33.848248
4  Noah Natude  Tomatoes  2020-02-10     0.209112     5.515429


In [ ]:
def get_soil_moisture_trend(farmer_name):
    print(f"📡 Radar Soil Moisture Check: {farmer_name}")
    geom = get_farm_geometry(farmer_portfolio[farmer_name][0]['coords'])

    # Sentinel-1 Radar (VV band is sensitive to soil moisture)
    s1 = (ee.ImageCollection('COPERNICUS/S1_GRD')
          .filterBounds(geom)
          .filterDate('2024-12-01', '2025-01-30')
          .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV'))
          .median())

    # Calculate a simplified moisture index (backscatter intensity)
    stats = s1.select('VV').reduceRegion(ee.Reducer.mean(), geom, 10).getInfo()
    vv_val = stats.get('VV')

    # Logic: Lower (more negative) values usually mean drier soil
    if vv_val < -15:
        return "🚨 ALERT: Soil is extremely dry at the root zone."
    else:
        return "💧 Soil moisture levels are currently adequate."



In [ ]:
print(get_soil_moisture_trend("George Mukama"))

📡 Radar Soil Moisture Check: George Mukama
💧 Soil moisture levels are currently adequate.


In [ ]:
def get_disease_risk(farmer_name):
    print(f"🔍 Scanning {farmer_name}'s farm for Biological Stress...")
    geom = get_farm_geometry(farmer_portfolio[farmer_name][0]['coords'])

    # Get recent imagery
    image = (ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')
             .filterBounds(geom)
             .filterDate('2024-12-01', '2025-01-30')
             .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20))
             .median())

    # Calculate NDWI (Leaf Water Content)
    ndwi = image.normalizedDifference(['B8', 'B11']).rename('NDWI')

    # Calculate Humidity/Temp Risk (Fungal diseases love high humidity)
    # We use CHIRPS as a proxy for leaf wetness
    rain = ee.ImageCollection('UCSB-CHG/CHIRPS/DAILY').filterDate('2025-01-01', '2025-01-30').sum()
    recent_rain = rain.reduceRegion(ee.Reducer.mean(), geom, 5000).getInfo().get('precipitation')

    stats = ndwi.reduceRegion(ee.Reducer.mean(), geom, 10).getInfo()
    ndwi_val = stats.get('NDWI')

    # Logic for Alerts
    if ndwi_val < 0.1 and recent_rain < 20:
        return "🔴 PEST ALERT: High risk of Sucking Pests (Aphids/Mites). Leaves are abnormally dry."
    elif ndwi_val > 0.4 and recent_rain > 100:
        return "🟠 DISEASE ALERT: High Fungal Risk (Blight/Rust). Conditions are too wet."
    else:
        return "🟢 STABLE: No immediate biological stress detected."

print(get_disease_risk("Noah Natude"))

🔍 Scanning Noah Natude's farm for Biological Stress...
🟢 STABLE: No immediate biological stress detected.


In [ ]:
def detect_pest_anomaly(farmer_name):
    print(f"📉 Checking for health anomalies for {farmer_name}...")
    geom = get_farm_geometry(farmer_portfolio[farmer_name][0]['coords'])

    # 1. Current Health (Last 30 days)
    current_ndvi = (ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')
                   .filterBounds(geom).filterDate('2025-01-01', '2025-01-30')
                   .median().normalizedDifference(['B8', 'B4'])
                   .reduceRegion(ee.Reducer.mean(), geom, 10).getInfo().get('nd'))

    # 2. Historical Baseline (Same month, previous 3 years)
    baseline_ndvi = (ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')
                    .filterBounds(geom).filterDate('2021-01-01', '2024-01-30')
                    .filter(ee.Filter.calendarRange(1, 1, 'month')) # Only Januaries
                    .median().normalizedDifference(['B8', 'B4'])
                    .reduceRegion(ee.Reducer.mean(), geom, 10).getInfo().get('nd'))

    if current_ndvi and baseline_ndvi:
        diff = (current_ndvi / baseline_ndvi)
        if diff < 0.80: # 20% drop compared to history
            return f"🚨 CRITICAL ANOMALY: Farm health is {((1-diff)*100):.1f}% BELOW normal. Scouting for Pests is required!"
        else:
            return "✅ NORMAL: Farm is performing within historical ranges."
    return "Data unavailable for comparison."



In [ ]:
print(detect_pest_anomaly("Noah Natude"))

📉 Checking for health anomalies for Noah Natude...
✅ NORMAL: Farm is performing within historical ranges.
